# Introduction to Pandas
Pandas is an extensive, powerful library for manipulating and analyzing serial and/or tabular data structures with column and row labels. It comes with a suite of built-in statistical functions, time series construction and analysis tools, grouping and sorting functions, hierarchical indexing functions, and even some built-in plotting methods. The data structures that Pandas is built around are highly mutable, adaptable to many data types, and can be loaded from or saved to a wide variety of text, binary, or database files. Several of Python's machine learning libraries and large language modeling packages are built around pandas data structures, especially SciKitLearn and TensorFlow. [Some accelerated operations are supported using the `bottleneck` and `numexpr` libraries.](https://pandas.pydata.org/docs/user_guide/basics.html#accelerated-operations)

In the interest of ensuring you know about the best tools for your data, I should mention that Pandas has a couple of potentially deal-breaking limitations: support for data structures with >2 dimensions is limited and hard to use, and there is little native support for parallelization. If you have N-dimensional data where N>2, [`Xarray` is probably the better choice of software](https://xarray.dev/). Similarly, if speed is enough of an issue that you need to distribute your array processing over multiple CPUs or nodes, including `bottleneck` and `numexpr` may help, but [you might consider `Polars` instead](https://docs.pola.rs/).

In [1]:
import pandas as pd
import numpy as np
#import seaborn as sb 
###^standard import abbr is sns, after a West Wing character's initials
###I use sb so I don't have to remember that. I never saw West Wing.
#import matplotlib.pyplot as plt
#import matplotlib as mpl
#%matplotlib inline

### Pandas Object Classes
Most data structures you will use in Pandas will be one of the following two object classes:

1. `Series`: a 1D array that can optionally have an "index" assigned to every entry and a "name" for the whole distinct from the variable name in your code. The indexes (and name) need only to be hashable; they don't have to be numeric or even unique. Data can also be of any type.
2. `DataFrame`: a 2D array or table of values where every row and column can be assigned a label and/or index; like a `dict` of `Series` stacked column-wise, but with different data selection syntax. Arithmetic operations can be performed along either axis, where axis=0 typically refers to row-wise operations and axis=1 refers to column-wise operations. The data type of each column is determined separately. DataFrames can also be grouped by column or row values with the `groupby()` method, or rearranged with the `pivot()` or `pivot_table()` methods, such that the result is hierarchical, like a 2D projection of higher-dimensional data structures.

These are the workhorses of data analysis with Pandas. Other Pandas data structures you may encounter include:

3. `Index`: the Pandas datatype of attributes that store the row and column labels of a DataFrame or Series. You could convert a 1D array to an `Index` type object, but more often you will extract `Index` objects from existing Series or DataFrames and, if needed, convert them to NumPy arrays or lists.
4. `DataFrameGroupBy` (abbr. GroupBy): a structure returned by the `df.groupby()` method. It may look like the input DataFrame sorted by the values of the column label(s) entered as arguments, but for any basic function or statistical method called on this object, the number of return values will equal the number of unique values or value combinations from the column(s) used to group the DataFrame contents. If two or more labels are used for grouping, the result will be hierarchical, and will have MultiIndexes instead of just Indexes.
5. `MultiIndex`: the hierarchical analogue of `Index` for GroupBy objects and other hierarchical DataFrames. Each list item is a tuple instead of a string or scalar value. We will talk about MultiIndexing and hierarchical DataFrames rather late in this course because, in my view, their complexity makes their usage awkward and error-prone. If you need such functionality and don't like the constraints of working strictly in 2D, Xarray might be a better alternative.
6. `NumpyExtensionArray`, `DatetimeArray`, and `TimedeltaArray`: to extend NumPy functionality, especially to time series, Pandas includes an `array(data, dtype=dtype)` function that can be used to declare simple arrays or store timestamps or time increments depending on what you enter for the `dtype` kwarg. Most familiar datatypes will produce an array of the `NumpyExtensionArray`, but if you set `dtype` equal to `'datetime64[ns]'` or `'timedelta64[ns]'`, the resulting array will be of `DatetimeArray` or `TimedeltaArray` type, respectively. 

DataFrames are the primary structure that Pandas is designed to work with, so we'll focus on those. **Functions that work on a DataFrame will also work on a Series unless they explicitly require multiple columns.** For the sake of demonstration, however, I will mock up a couple of example Series and DataFrames.

Series construction is simple: just call `pandas.Series()`. It is best if you can provide at least a 1D data array/list, if not also a corresponding list of indexes—for the same reason that allocating and then replacing values in a NumPy array is faster than appending to a list—but it is possible to create an empty Series and fill it later. If you provide the data but no index list, indexes will be assigned automatically.

In [308]:
ser1 = pd.Series(['a','b','c'], index=[1,2,3], name='abc')
print(ser1)

1    a
2    b
3    c
Name: abc, dtype: object


Note that the left column changes depending on whether we include the index kwarg (default index is 0-based), and that you can optionally assign the series a name.

The indexes also do not have to be numeric. They can be letters or even datatime objects.

In [10]:
ser2 = pd.Series([1,2,3], index=['a','b','c'], name='abc123')
print(ser2)

a    1
b    2
c    3
Name: abc123, dtype: int64


In [23]:
#from datetime import datetime as dttm
sunrises= ['2023-12-21 08:34:00',
           '2023-12-31 08:36:00',
           '2024-01-11 08:31:00']
daylens = [7.01583, 7.11306, 7.46278]
srt = pd.to_datetime(sunrises,
                     format="%Y-%m-%d %H:%M:%S")
#much easier to use pandas to convert to datetime objects than
# to go through the datetime module - more on this later
print(srt)
print(pd.Series(daylens,index=srt))

DatetimeIndex(['2023-12-21 08:34:00', '2023-12-31 08:36:00',
               '2024-01-11 08:31:00'],
              dtype='datetime64[ns]', freq=None)
2023-12-21 08:34:00    7.01583
2023-12-31 08:36:00    7.11306
2024-01-11 08:31:00    7.46278
dtype: float64


Construction of a DataFrame is similar to that of a Series, but now you have the option to specify both the column names and the row indexes. If you leave these kwargs empty, pandas will assign 0-based integer indexes to both. Technically, even the first positional argument, the data, is optional. You can leave it blank and fill it later, although this is not recommended (see sub-section on initializing DataFrames with NumPy vs. pure Pandas).

In [305]:
dummy_df = pd.DataFrame(np.linspace(0.5,10,20).reshape(5,4),
                        columns=['a','b','c','d'], index=list(range(1,6)))
print(dummy_df)

     a    b    c     d
1  0.5  1.0  1.5   2.0
2  2.5  3.0  3.5   4.0
3  4.5  5.0  5.5   6.0
4  6.5  7.0  7.5   8.0
5  8.5  9.0  9.5  10.0


#### Basic Attributes
DataFrames and Series have more than a dozen attributes besides those used to select subsets of the data, and several hundred object methods to transform, aggregate, and broadcast data and functions thereof. We cannot possibly cover all the object methods, but I will do my best to cover the ones I have experience with and/or can demonstrate within the limits of the presentation format.

There are a small enough number of attributes that they can be listed here and in the upcoming section on data selection:
| Attribute Syntax | Value |
| --- | --- |
| `df.axes` | nested list of row & column indexes (labels) |
| `df.columns` | Index-type, list of column labels (add `.values` to get an array) |
| `df.index` | Index-type, list of row indexes (labels; add `.values` to get an array) |
| `df.dtypes` | list of datatypes by column |
| `df.empty` | boolean, True if df is empty |
| `df.ndim` | number of axes (1 for a Series, 2 for a DataFrame) |
| `df.shape` | tuple, length of df along each axis |
| `df.size` | integer, total number of data entries |
| `df.values` | returns df converted to a NumPy array (can also be applied to `.columns` & `.index`) |

Here are a few of the most basic attributes in action:

In [301]:
dummy_df.axes #returns both row and column labels/indexes in a nested list

[Index([1, 2, 3, 4, 5], dtype='int64'),
 Index(['a', 'b', 'c', 'd'], dtype='object')]

In [302]:
print(dummy_df.ndim, dummy_df.shape, dummy_df.size)

2 (5, 4) 20


In [286]:
print('T/F - Series', ser1.name, 'contains NaNs:', ser1.hasnans) #Series only

T/F - Series abc contains NaNs: False


#### Initializing DataFrames with NumPy vs. Pure Pandas
If you have to build a DataFrame from scratch within a Python program (e.g. as the output of a bunch of simulations), allocating and filling a NumPy array and then converting to a DataFrame is usually faster than creating an empty DataFrame and filling that, particularly if the NumPy array is purely comprised of floats. It's up to you to determine the best balance of speed and fool-proofing depending on the method you use to fill your (future) DataFrame, and you may need to test subsets of your data to find the best construction method depending on how you assign entries, rows, or columns to the DataFrame.

Let's say you're simulating 100 dust grains in an interstellar UV radiation field over 10000 hours, and the following DataFrame is to keep track of the instantaneous surface temperature of each grain in Kelvin (something vaguely like this could show up in an astrochemistry paper). Let's compare a NumPy-first instantiation to pure Pandas. To make this demo a little more realistically inefficient to construct, I assume that each particle's data comes from a separate data structure and so must be added to the final array 1 column at a time. Note that I'm using one of several time series constructor functions.

In [111]:
import time
partid = np.arange(1,101)
ts = pd.timedelta_range(start='1 day',periods=10000, freq='1H')
print(ts[:30])

TimedeltaIndex(['1 days 00:00:00', '1 days 01:00:00', '1 days 02:00:00',
                '1 days 03:00:00', '1 days 04:00:00', '1 days 05:00:00',
                '1 days 06:00:00', '1 days 07:00:00', '1 days 08:00:00',
                '1 days 09:00:00', '1 days 10:00:00', '1 days 11:00:00',
                '1 days 12:00:00', '1 days 13:00:00', '1 days 14:00:00',
                '1 days 15:00:00', '1 days 16:00:00', '1 days 17:00:00',
                '1 days 18:00:00', '1 days 19:00:00', '1 days 20:00:00',
                '1 days 21:00:00', '1 days 22:00:00', '1 days 23:00:00',
                '2 days 00:00:00', '2 days 01:00:00', '2 days 02:00:00',
                '2 days 03:00:00', '2 days 04:00:00', '2 days 05:00:00'],
               dtype='timedelta64[ns]', freq='H')


In [129]:
#Start with NumPy implementation
runtimes = np.zeros(100)
for i in range(100):
    t0 = time.time_ns()
    data = np.zeros((len(ts), len(partid)))
    for j,v in enumerate(partid):
        data[:,j] = 10*np.random.chisquare(3,len(ts))
        #this gives fairly believably values for such a quick and dirty demo
    dummy_df = pd.DataFrame(data,columns=partid,index=ts)
    if i==1:
        print(dummy_df.head())
    runtimes[i] = (time.time_ns()-t0)
    del data #trying to stop NumPy from skewing the results with cached data
    del dummy_df
    #doesn't seem to help
print('NumPy-first approach took {:.3f} ms'.format(np.mean(runtimes)*10**-6))
print(runtimes[:10]*10**-6)

                        1          2          3          4          5    \
1 days 00:00:00   45.132719  18.706452  23.681400  77.297622  29.968109   
1 days 01:00:00   42.414091  15.198667   8.644502  26.829536   4.930699   
1 days 02:00:00   87.395789  22.426079  85.479914  17.824291  13.695420   
1 days 03:00:00  150.882766  39.727747   4.926684  35.183574   7.326537   
1 days 04:00:00   11.864747  40.819665   8.266819  31.063394  20.859097   

                       6          7          8          9          10   ...  \
1 days 00:00:00  13.487936  10.816268  27.426948  11.739450  22.612319  ...   
1 days 01:00:00  57.313018  86.826926  21.660628   5.241199  19.713096  ...   
1 days 02:00:00  43.080175  10.783574  67.239200  64.652637  16.542422  ...   
1 days 03:00:00  15.930251   7.082038  76.137979  15.832700  59.018916  ...   
1 days 04:00:00  10.197294  24.008260  15.544460   6.295166  13.939077  ...   

                        91         92         93         94         95   \

In [132]:
#now the pure Pandas approach
runtimes = np.zeros(100)
for i in range(len(runtimes)):
    t0 = time.time_ns()
    dummy_df2=pd.DataFrame(columns=partid,index=ts)
    for pid in partid:
        dummy_df2[pid]=10.*np.random.chisquare(3,len(ts))
    if i==1:
        print(dummy_df2.head())
    runtimes[i] = (time.time_ns()-t0)
    del dummy_df2  #try to avoid skewing the results with cached data
    #doesn't seem to help
print('Pure Pandas approach took {:.3f} ms'.format(np.mean(runtimes)*10**-6))
print(runtimes[:10]*10**-6)

                       1          2          3          4          5    \
1 days 00:00:00  35.313166  11.996560  62.050499  23.940570  34.210093   
1 days 01:00:00  33.586879  23.129506  21.573475   6.287035  26.682972   
1 days 02:00:00  24.858662   3.823288  94.689335  40.610323  36.974038   
1 days 03:00:00  12.265710  22.941618  21.732078  35.709924  53.274873   
1 days 04:00:00   1.740359  27.318770  55.228072  45.982285  28.161902   

                       6          7          8          9          10   ...  \
1 days 00:00:00   9.948517   3.366698   2.798905  40.368338  16.360232  ...   
1 days 01:00:00  24.520840  35.693240  16.916476  66.463949  55.664016  ...   
1 days 02:00:00  69.898572  12.921330  26.019181  30.696951  42.910638  ...   
1 days 03:00:00  25.931199  43.234775  11.747854  79.388570  66.596115  ...   
1 days 04:00:00  30.965781  20.444539  48.600102  15.508818  27.792224  ...   

                       91         92         93         94          95   \
1 day

As you can see, the NumPy-first approach can do about 4 iterations of the program in the time it takes the pure-Pandas version to do about 3 iterations. You can also see that the first attempt (or 2) to run either set of calculations usually takes longer than subsequent calculations despite my best efforts to eliminate cached data, though this is more consistently true for the Pandas approach.

**Note on converting DataFrames to Numpy.** If you need to convert a DataFrame to a numpy array, you can use `df.to_numpy()`. However, both column and index labels will then be lost. If you used a data column as indexes and want to keep it, you can append the indexes as a column before converting to an array, but remember that NumPy arrays have limited support for mixed data types.

### Basic I/O
The most used read/write combos are `df = pd.read_csv()`/`df.to_csv()` and `df = pd.read_excel()`/`df.to_excel()`, but Pandas can read and write to a wide variety of text and binary formats, including HDF5 and Python pickle files. Even SQL tables can be loaded or output. [I'll refer you to the documentation for the full list of readers and writers](https://pandas.pydata.org/docs/user_guide/io.html#). Most of these readers also accept a URL as a filepath if you want to pull data from a public online repository like Kaggle.

**Most pandas data readers default to assuming the top row is a row of column names**. If you have data where you know this not to be the case, you can either choose a different row by setting the `header` kwarg to a different number or override the column names by setting `columns` equal to a list of your choosing. For row indexes/labels, **the default behavior of most pandas data readers is to assume there is no index column and assign 0-based indexes to every row below the header row.** Often there is a column in the dataset that makes more sense to index rows by, so you can change the default behavior by setting `index_col` equal to index of the column you want to use as row indexes. Remember that columns are indexed left to right starting from 0.

The table in the documentation does not include it, but for standard tab-delimited text files, you can use either `pd.read_table(filepath_or_buffer,sep='\t')` or `pd.read_csv(filepath_or_buffer,sep='\t')`. Many of the keyword arguments (kwargs) in these and other reader functions are similar to those of `np.gen_from_txt()`, but are much less fussy about text encoding, missing values, and mixed data types. 

I'll load one of my favorite datasets, the 5250 exoplanet dataset. Those of you who attended my Matplotlib for Publication workshop will remember this from the exercises. (I like it because it's very clean for real data but it's big enough and just realistically flawed/"ugly" enough to provide some practice at data cleanup.) Notice that I've kept the default behavior for the column names, but have changed the row indexes to the `name` column, which was the leftmost column in the csv file. It didn't make sense to assign a number to every planet when each planet comes with a more meaningful and unique identifier.

In [115]:
df = pd.read_csv('exoplanets_5250_EarthUnits.csv',index_col=0)
df.head()

,distance,star_mag,planet_type,discovery_yr,mass_ME,radius_RE,orbital_radius_AU,orbital_period_yr,eccentricity,detection_method
#name,,,,,,,,,,
11 Comae Berenices b,304.0,4.72307,Gas Giant,2007,6169.20,12.096,1.290000,0.892539,0.23,Radial Velocity
11 Ursae Minoris b,409.0,5.01300,Gas Giant,2009,4687.32,12.208,1.530000,1.400000,0.08,Radial Velocity
14 Andromedae b,246.0,5.23133,Gas Giant,2008,1526.40,12.88,0.830000,0.508693,0.00,Radial Velocity
14 Herculis b,58.0,6.61935,Gas Giant,2002,2588.14,12.544,2.773069,4.800000,0.37,Radial Velocity
16 Cygni B b,69.0,6.21500,Gas Giant,1996,566.04,13.44,1.660000,2.200000,0.68,Radial Velocity


The columns of this data table are, from left to right:
- **name**: exoplanet name
- **distance**: distance to the planetary system in light years
- **star_mag**: apparent brightness of the star as seen from Earth in the astronomical magnitude system (which is admittedly awful)
- **planet_type**: values include "Gas Giant" (like Jupiter or Saturn), "Neptune-like" (sort of a mini gas giant or ice giant), "Super Earth" (thought to be rocky but up to a few times larger than Earth), and "Terrestrial" (comparable to or smaller than Earth)
- **discovery_yr**: year that the discovery of the planet was published
- **mass_ME**: mass of the planet in units of Earth masses (1 Earth mass = $5.972 \times 10^{24}$ kg)
- **radius_RE**: radius of the planet in units of Earth radii (1 Earth radius = 6371 km)
- **orbital_radius_AU**: the exoplanet's orbital semi-major axis in units of the average distance between the Earth and Sun (1 Astronomical Unit, or AU)
- **orbital_period_yr**: the time taken for the exoplanet to orbit its star in units of Earth's orbital period (1 year)
- **eccentricity**: measure of the deviation of the exoplanet's orbit from a perfect circle. Values range from 0 to 1 where 0 = perfect circle and 1 indicates a parabolic (just barely unbound) orbit.
- **detection_method**: principle method used to detect the planet. See [this educational slideshow by NASA](https://exoplanets.nasa.gov/alien-worlds/ways-to-find-a-planet/) or [this Wikipedia page](https://en.wikipedia.org/wiki/Methods_of_detecting_exoplanets) for more details.

When I loaded the DataFrame above, note that I set `index_col=0`, that is, I told `read_csv()` to use the leftmost column as the indexes of the DataFrame. That means I can now grab the entries of any planet by name, if I know the name of the planet as it is rendered in this database. Without that command, `'#name'` is just another column, and every row is assigned a numerical index in the order in which it appears, starting at 0. The default can be useful for regularly sampled time series, but these data  The default behavior for setting column names is to assume the first row is a row of column labels, so I did not need to do anything else. If your data have column labels on another row, `read_csv()` also has the `header` and `skiprows` kwargs to specify the row with the column names and rows to ignore, respectively. The list of other kwargs for parsing and formatting data, and managing memory while you do it, is very long, so [I will refer you to the onliner documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv) and move on.

If for some reason I wanted to, for example, save this dataframe as a plain text file with pipes (|) for separators and commas in place of decimal points (don't do this with regular CSV files!), I would use:

In [26]:
df.to_csv('exoplanets_5250_EarthUnits.txt', sep='|',
          decimal=',', index=True)

Note: the index kwarg is `True` by default, but I draw it to your attention because if you allow default indexing and don't want those indexes to be saved, you may want to set that kwarg to `False`.

If I wanted to append the table as a new sheet in an existing Excel file, the command would be something like:
`df.to_csv('other_file.xlsx', mode='a', sheet_name='Sheet2')`

We will go more in depth into I/O with more complex and hierarchical data later.

### Inspecting Data
Recall that the first thing I did after importing was call this function `df.head()`. That's a good way to get an overview of your data without loading the whole thing: it displays the first 5 rows of the table with all column names and row indexes. For tables larger than 2 GB, your first view of the data is likely to be with `df.head()` simply because Excel and other text editors/viewers may refuse to load a file of that size. There is also a `df.tail()` function that outputs the last 5 rows of your data instead. Both `head()` and `tail()` accept an integer argument for the number of rows to return if you want a different number than the default 5. E.g. if you only wanted the last row, you would call `df.tail(1)`.

In [76]:
df.tail(3)

,distance,star_mag,planet_type,discovery_yr,mass_ME,radius_RE,orbital_radius_AU,orbital_period_yr,eccentricity,detection_method
#name,,,,,,,,,,
YZ Ceti b,12.0,12.074,Terrestrial,2017,0.70,0.913,0.01634,0.005476,0.06,Radial Velocity
YZ Ceti c,12.0,12.074,Super Earth,2017,1.14,1.05,0.02156,0.008487,0.00,Radial Velocity
YZ Ceti d,12.0,12.074,Super Earth,2017,1.09,1.03,0.02851,0.012868,0.07,Radial Velocity


If you have a smaller or more symmetrical table that might be more intuitively reorganized if the row and column orders were switched, you can transpose the table with `df.T`.

In [157]:
dummy = pd.DataFrame(np.linspace(0.5,10,20).reshape(5,4),columns=['a','b','c','d'])
print(dummy,'\n\n',dummy.T)

     a    b    c     d
0  0.5  1.0  1.5   2.0
1  2.5  3.0  3.5   4.0
2  4.5  5.0  5.5   6.0
3  6.5  7.0  7.5   8.0
4  8.5  9.0  9.5  10.0 

      0    1    2    3     4
a  0.5  2.5  4.5  6.5   8.5
b  1.0  3.0  5.0  7.0   9.0
c  1.5  3.5  5.5  7.5   9.5
d  2.0  4.0  6.0  8.0  10.0


Other great tools for getting an overview (and sanity check) of your data are `df.info()` and `df.describe()`. `df.info()` prints the zero-based index, name, count of non-Null data, and the data type of each colum for all columns, and also briefly describes the row-indexing system and the size of the DataFrame in memory. `df.describe()` immediately outputs the count, mean, standard deviation, minimum, maximum, and quartiles of all numeric columns, automatically excluding NaNs. **Note:** integer columns are treated as floats and object-type columns are ignored even if most of the data are numeric.

In [338]:
df.describe()

,distance,star_mag,discovery_yr,mass_ME,radius_RE,orbital_radius_AU,orbital_period_yr,eccentricity
count,5233.000000,5089.000000,5250.000000,5227.000000,5233.000000,4961.000000,5.250000e+03,5250.000000
mean,2167.168737,12.683738,2015.732190,460.035267,5.627083,6.962942,4.791509e+02,0.063924
std,3245.522087,3.107571,4.307336,3761.458727,5.315522,138.673600,1.680445e+04,0.141402
min,4.000000,0.872000,1992.000000,0.020000,0.296000,0.004400,2.737850e-04,0.000000
25%,389.000000,10.939000,2014.000000,3.970000,1.760000,0.053000,1.259411e-02,0.000000
50%,1371.000000,13.543000,2016.000000,8.470000,2.732800,0.102800,3.449692e-02,0.000000
75%,2779.000000,15.021000,2018.000000,159.000000,11.715200,0.286000,1.442163e-01,0.060000
max,27727.000000,44.610000,2023.000000,239136.000000,77.280000,7506.000000,1.101370e+06,0.950000


In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5250 entries, 11 Comae Berenices b to YZ Ceti d
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   distance           5233 non-null   float64
 1   star_mag           5089 non-null   float64
 2   planet_type        5250 non-null   object 
 3   discovery_yr       5250 non-null   int64  
 4   mass_ME            5250 non-null   object 
 5   radius_RE          5250 non-null   object 
 6   orbital_radius_AU  4961 non-null   float64
 7   orbital_period_yr  5250 non-null   float64
 8   eccentricity       5250 non-null   float64
 9   detection_method   5250 non-null   object 
dtypes: float64(5), int64(1), object(4)
memory usage: 451.2+ KB


The count lists of both descriptive functions immediately tell me that most columns have some missing data, and the data types for `mass_ME` and `radius_RE` are a red flag. The metrics returned by `.describe()` also tell me, given some subject-matter expertise, that at least half of the eccentricity values should be taken as assumed filler values (i.e. with a mountain of salt). In a previous iteration I also saw negative values in that column, which are physically impossible and indicated a need to update or nullify those values.

**Memory Usage.** Another stat that you should take with a mountain of salt: the memory usage stat. Since most lecture attendees are HPC users, let's take a closer look with a function that many of you will need if you plan to use Pandas on NAISS resources: `.memory_usage()`.

In [178]:
df.memory_usage()#deep=True)

Index                174136
distance              42000
star_mag              42000
planet_type           42000
discovery_yr          42000
mass_ME               42000
radius_RE             42000
orbital_radius_AU     42000
orbital_period_yr     42000
eccentricity          42000
detection_method      42000
dtype: int64

This function returns the size in memory of every column, plus the size of the indexes by default. You can hide the contribution of the row and column labels by setting `index=False`, and you can get an estimate of how much system-level memory the object-type data columns consume by setting `deep=True`. The latter is important to check because object-type columns can be much larger in memory than initially reported. Watch what happens when the command is rerun with `deep=True` uncommented.

What's going on? [This article](https://pythonspeed.com/articles/pandas-dataframe-series-memory-usage/) provides a fuller and perhaps better explanation, but the short answer is that numerical and Boolean datatypes have fixed size in memory (e.g. an int64 or float64 number will always be 8 bytes, whether it's 1 or Avogadro's number), whereas object-type data (strings) are variable in size and usually must be stored somewhere else because they're almost always bigger. When memory is allocated for the DataFrame itself, that parcel of memory also contains all of the numerical or Boolean data, but only pointers to the object-type data. The actual values of the object-type data are stored wherever space can be found in memory, which requires significantly more overhead. When `deep=False`, as is the default, *the memory usage reported for object-type data is only what is used by the pointers*.

That said, even when `deep=True`, you only get a *worst-case estimate* of the memory usage of object-type data. The actual usage as reported by a dedicated memory profile will typically be somewhat smaller because Python has some built-in string optimization routines that cache frequently used strings, and because the estimate often includes temporary structures that get deallocated. Still, if `.memory_usage()` is your only tool to check the size of your data in memory, the results when `deep=True` will generally be closer to the real value than the default output (and it's better to design around the worst-case scenario).

Back to basics: some of the other returns of `.info()` can also be called individually for the whole DataFrame or individual columns or rows, specifically `.count()` and, as we mentioned in the attributes section, `.dtypes`. Whether you need the count(s) for the whole DataFrame or just one row or column, with the correct data selection syntax, the method is just `.count()`.

In [243]:
df.loc['55 Cancri e'].count()

10

That basically means the row labelled '55 Cancri e' has 10 data entries. If you suspect there are duplicates, you can replace `.count()` with `.nunique()`, which will return a single value for a Series or Index list, and a Series when given a DataFrame.

In [337]:
print('Of', df['eccentricity'].count(), 'eccentricity values, only', 
      df['eccentricity'].nunique(), 'are unique.')

Of 5250 eccentricity values, only 175 are unique.


There is also a `.value_counts()` method that counts every unique row-wise combination of values for however many columns you give it. It is mainly used for GroupBy objects, which we will discuss later. For now, just observe the behavior below:

In [83]:
print(df['detection_method'].value_counts())
print('\nCompare:\n')
print(df[['discovery_yr', 'detection_method']].value_counts())

detection_method
Transit                          3945
Radial Velocity                  1027
Gravitational Microlensing        154
Direct Imaging                     62
Transit Timing Variations          24
Eclipse Timing Variations          17
Orbital Brightness Modulation       9
Pulsar Timing                       7
Astrometry                          2
Pulsation Timing Variations         2
Disk Kinematics                     1
Name: count, dtype: int64

Compare:

discovery_yr  detection_method           
2016          Transit                        1453
2014          Transit                         802
2021          Transit                         415
2018          Transit                         253
2022          Transit                         188
                                             ... 
2013          Astrometry                        1
2007          Pulsation Timing Variations       1
              Direct Imaging                    1
2018          Transit Timing Variati

To get the datatypes of a single column, a singular version of the datatypes attribute is used, `df['col_name'].dtype` because column data are expected to all be of the same datatype. For a row, which is expected to have varying data types, the syntax is more like that used for the whole DataFrame: `df.loc['row_name'].dtypes` or `df.iloc['row_index'].dtypes` depending on whether you select the row by a custom label or the 0-based numerical index, respectively.

In [17]:
df.dtypes

distance             float64
star_mag             float64
planet_type           object
discovery_yr           int64
mass_ME               object
radius_RE             object
orbital_radius_AU    float64
orbital_period_yr    float64
eccentricity         float64
detection_method      object
dtype: object

As for why the mass and radius columns have type `object` instead of `float64`, let's see what happens if we try to coerce the data of those columns to the expected type (don't worry about the syntax just yet):

In [245]:
df['mass_ME'].astype('float64')

ValueError: could not convert string to float: ' '

Aha. There's at least one whitespace in that column. Pandas was initially designed for economic data where cells might contain dates, currency symbols, addresses, and other types of data where numbers might appear with spaces, so Pandas assumes all whitespace is intentional and marks the column as `object` even if the whitespace is in an otherwise totally numerical column. Pandas will only coerce columns of numbers and missing values to `float64` if there are no non-numeric characters.

Prior inspection of the data revealed that the mass and radius columns both had a mix of floats, integers (as a result of rounding to significant digits), missing values, and *cells that look empty but actually contain a space character*. Let's take a quick look at an entry where I know this has happened from looking at the original Excel file:

In [248]:
print(df.loc['Kepler-97 c'])

distance                      1308.0
star_mag                      12.994
planet_type                Gas Giant
discovery_yr                    2014
mass_ME                       343.44
radius_RE                           
orbital_radius_AU                NaN
orbital_period_yr                2.2
eccentricity                     0.0
detection_method     Radial Velocity
Name: Kepler-97 c, dtype: object


In [250]:
df.loc['Kepler-97 c','radius_RE']

' '

If I select the whole row or a subset of the columns, the value in the radius column just looks empty, but if I look at just the *'radius_RE'* column for this planet, it shows the single space in quotes.

Pandas devotes a substantial fraction of its functional library to making it easy to deal with malformed data. An issue like this could be fixed at import with the `converters` kwarg of `read_csv()` if you know about it a priori. I held off so I can show you a little later how to find and handle different types of missing data within Python, as you might if your typical choice of file viewer chokes on the size of the data.

To get there, we need to dive into how subsets of Pandas DataFrames are selected, so you can understand the use of `df.loc[...]` and `df.iloc[...]` among other selection methods.

### Accessing Data
Access by column(s) and/or row(s) is simple in principle, though it can be error prone if you're used to NumPy arrays and Python dictionaries. The official documentation can be a bit verbose if you're just trying to remember when to use `.loc`, `.iloc`, or neither. The following summary table should help:

| To Access... | Syntax |
| :--- | :--- |
| 1 column | `df['col_name']` |
| 1 named row | `df.loc['row_name']` |
| 1 row by index | `df.iloc[index]` |
| 1 column by index (rarely used) | `df.iloc[:,index]` |
| subset of columns | `df[['col0', 'col1', 'col2']]` |
| subset of named rows | `df.loc[['rowA','rowB','rowC']]` |
| subset of rows by index | `df.iloc[i_m:i_n]`  where *i_m* & *i_n* are the m<sup>th</sup> & n<sup>th</sup> integer indexes |
| rows & columns by name | `df.loc['row','col']` or `df.loc[['rowA','rowB', ...],['col0', 'col1', ...]]` |
| rows & columns by index | `df.iloc[i_m:i_n, j_p:j_q]` where *i* & *j* are row & column indexes, respectively |
| columns by name & rows by index | `df[['col0', 'col1', 'col2']].iloc[i_m:i_n]` |

**Columns** alone can be selected by name in square brackets (`[]`) like ordinary dictionary entries. **Rows**, with or without columns, must be accessed by adding either `.loc` if selection is by name, or `.iloc` if selection is by index, between the name of the DataFrame and the `[]`. If you need to select both rows and columns, the row and column names must be given in row-major order, as with most other Python array functions: `[row(s), col(s)]`. Also note that `.iloc[]` is endpoint-exclusive like regular Python array slicing operations, while `.loc[]` is endpoint-inclusive. 

As a reminder, if you just want to view the column labels or row labels/indexes, the the commands are `df.columns` and `df.index`, respectively. Both return Pandas Series of type `Index`, which can be used with any Pandas method that works on Series but *cannot* be directly input into NumPy functions or list comprehension. If you need the output of either command to be a list or an array, add `.values`.

In [176]:
print(df.iloc[1])

distance                       409.0
star_mag                       5.013
planet_type                Gas Giant
discovery_yr                    2009
mass_ME                      4687.32
radius_RE                     12.208
orbital_radius_AU               1.53
orbital_period_yr                1.4
eccentricity                    0.08
detection_method     Radial Velocity
Name: 11 Ursae Minoris b, dtype: object


In [186]:
print(df[['planet_type','mass_ME']].iloc[25:35])

                planet_type  mass_ME
#name                               
51 Eridani b      Gas Giant   636.00
51 Pegasi b       Gas Giant   146.28
55 Cancri b       Gas Giant   264.13
55 Cancri c       Gas Giant    54.51
55 Cancri d       Gas Giant  1233.20
55 Cancri e     Super Earth     7.99
55 Cancri f       Gas Giant    44.84
61 Virginis b  Neptune-like     5.10
61 Virginis c  Neptune-like    18.20
61 Virginis d  Neptune-like    22.90


#### Conditional Data Selection
In practice, you will very often need to filter rows (and columns) by conditions rather than names or indexes. Conditional operators (`==`, `>`, `<`, `=>`, `=<`, and `!=`) return boole-type Pandas Series that are used to select and return subsets of the input data for which the condition is `True`. For a single condition, the syntax is fairly intuitive once you've memorized the previous syntax table. To filter by multiple conditions, however, there are a few extra things to remember: 
1. To combine multiple conditions, you must use the "bitwise or" pipe operator `|` and the "bitwise and" ampersand operator `&`, instead of the usual `or` or `and`, respectively. The "bitwise exclusive or" operator `^` and "bitwise not" operator `~` are also available.
2. Each condition must be enclosed in parentheses `()` so that all conditions will be evaluated to boole-type Pandas Series that the bitwise operators can safely combine. Typically, forgetting the `()` will show up as a TypeError since conditions are often combined across differently-typed columns, but if you want to understand why errors result from missing `()`, it is helpful to know that the filtering expression `df['A']>2 & df['B']<=5` would be evaluated as `df['A']>(2 & df['B']<=5)`. (Believe me, if I knew why, I would submit a push request to change this behavior.)
3. If you want to filter by a list of values, it is better to use the `.isin()` attribute than a bitwise chain of conditions. Syntax: `df['col'].isin([value1, value2, ...])`. Note that this does *not* work the other way around, i.e. you cannot use `.isin()` to check if column, row, or cell entries contain a substring.

In [194]:
print(df.loc[df['discovery_yr'] < 1999].iloc[:, :5],'\n')

                        distance  star_mag  planet_type  discovery_yr  mass_ME
#name                                                                         
16 Cygni B b                69.0   6.21500    Gas Giant          1996   566.04
47 Ursae Majoris b          45.0   5.03352    Gas Giant          1996   804.54
51 Pegasi b                 50.0   5.45309    Gas Giant          1995   146.28
55 Cancri b                 41.0   5.95084    Gas Giant          1996   264.13
70 Virginis b               58.0   4.96808    Gas Giant          1996  2381.82
GJ 876 b                    15.0  10.16000    Gas Giant          1998   723.64
HD 168443 b                129.0   6.92122    Gas Giant          1998  2424.15
HD 187123 b                150.0   7.83000    Gas Giant          1998   166.31
HD 195019 b                123.0   6.87591    Gas Giant          1998  1265.64
HD 210277 b                 69.0   6.54348    Gas Giant          1998   410.22
HD 217107 b                 65.0   6.15500    Gas Gi

In [191]:
print(df.loc[ (df['discovery_yr'] < 2007) &
              (df['planet_type'] != 'Gas Giant'),
      'planet_type'])

#name
55 Cancri e              Super Earth
GJ 436 b                Neptune-like
GJ 581 b                Neptune-like
GJ 876 d                Neptune-like
HD 160691 d             Neptune-like
HD 190360 c             Neptune-like
HD 4308 b               Neptune-like
HD 49674 b              Neptune-like
HD 69830 b              Neptune-like
HD 69830 c              Neptune-like
HD 69830 d              Neptune-like
HD 99492 b              Neptune-like
OGLE-2005-BLG-169L b    Neptune-like
OGLE-2005-BLG-390L b    Neptune-like
PSR B1257+12 b           Terrestrial
PSR B1257+12 c           Super Earth
PSR B1257+12 d           Super Earth
Name: planet_type, dtype: object


In [196]:
print(df.loc[(df.index.str.contains('PSR')) &
             (df['discovery_yr'] < 2000), 'planet_type'])
print('\n...looks the same as...\n')
print(df[(df.index.str.contains('PSR')) &
         (df['discovery_yr'] < 2000)]['planet_type'])
print("\n...but only use the first version!")

#name
PSR B1257+12 b    Terrestrial
PSR B1257+12 c    Super Earth
PSR B1257+12 d    Super Earth
Name: planet_type, dtype: object

...looks the same as...

#name
PSR B1257+12 b    Terrestrial
PSR B1257+12 c    Super Earth
PSR B1257+12 d    Super Earth
Name: planet_type, dtype: object

...but only use the first version!


Here I'm touching on a rather complex topic: [chain indexing](https://pandas.pydata.org/docs/user_guide/indexing.html#why-does-assignment-fail-when-using-chained-indexing). We don't have time to get into this in depth, but the gist is that if it looks like you can access data either with `df[y][x]` or `df.loc[y,x]`, where *x* is a column or subset of columns and *y* is a row label or row-filtering condition, **you should always prefer the .loc[] format**. The syntax that looks like standard nested list or nested dict selection is called chain indexing, and with DataFrames, it's hard to know whether it will return a copy of the selected data or a view of the original data. It's also not always this obvious when you've used chained indexing, so Pandas is programmed to raise a `SettingWithCopy` warning to help you avoid the frustration.

I also want to call your attention to this snippet:
`df.index.str.contains('PSR')`
Here we took advantage of one of the more brilliant features of Pandas: **string vectorization**. Many string functions can be called on and broadcast to any Pandas Series of type `object` (`string`) or `Index` by adding the attribute `.str` and then a string method of your choice. You can also append `.str.method()`, where `method` is any string method, directly after another method that returns a Series with string-like contents, like the example above. The official documentation contains a helpful [summary table of allowed string operations](https://pandas.pydata.org/docs/user_guide/text.html#method-summary) and detailed discussions of how to use methods with multiple input/output options.

#### The .query() method
Pandas has a `.query()` method that can make conditional selection simpler and more readable. It works a bit like the built-in `eval()` and `exec()` functions (in fact, it uses these functions under-the-hood) in that it takes your filter conditions as a string, and allows you to use the plain-English versions\* (`and`, `or` etc.) of the bitwise binary operators discussed in the previous section. It lets you use the word `index` if you don't know the name of the index label you're looking for, and lets you filter either index or column\*\* values with the same syntax *as long as all column and index labels are unique.* You can even reference variables within the query statement by prefixing variable names with the `@` symbol.

\*Binary operations are not implemented for some data types. If the array is coerced to `ExtensionArray` type, the plain-English forms of certain binary operators (namely `is` and `is not`) may raise a `NotImplemented` error.

\*\*Note that you can both select and filter by rows, but can only filter by columns. Column selection must be made via chain indexing, which means the `.query()` method is not suitable for assigning values.

Let's rewrite the previous data selection command with the `.query()` function, and let the year vary.

In [226]:
y = 2009
df.query("index.str.contains('PSR') and \
        `discovery_yr` < @y")['planet_type']

#name
PSR B1257+12 b    Terrestrial
PSR B1257+12 c    Super Earth
PSR B1257+12 d    Super Earth
PSR B1620-26 b      Gas Giant
Name: planet_type, dtype: object


**Very important:** to distinguish column and row names from other strings within the query statement, you must bracket column and row labels with *grave accents* (\`\`) instead of single or double quotes.

It does not matter whether you use single quotes for strings within the query and double quotes for the whole statement, or vice-versa.

#### Finding and handling invalid data
Pandas provides a couple of convenience functions for selecting only invalid or only valid data from your DataFrame or any subset of it that is at least a Series: `.isna()` and `.notna()`. You may also see `.isnull()` and `.notnull()`, but these are aliases for `.isna()` and `.notna()`, respectively, and the use of the `na` versions is generally preferred over `null`. The `.isna()` method selects both NaNs and None values, but not $\pm$infinity and, as I mentioned earlier, not whitespaces in otherwise numerical columns. Infinite values and whitespaces-as-placeholders need to be replaced with NaN or None in order to take advantage of Pandas' built-in filters for invalid data.

In [10]:
df[df['orbital_radius_AU'].isna()].iloc[:5,2:7]

,planet_type,discovery_yr,mass_ME,radius_RE,orbital_radius_AU
#name,,,,,
CI Tauri b,Gas Giant,2019,3688.80,12.432,NaN
CoRoT-7 d,Neptune-like,2022,17.14,4.3008,NaN
DS Tucanae A b,Neptune-like,2019,413.40,5.7008,NaN
EPIC 201238110 b,Super Earth,2019,4.16,1.87,NaN
EPIC 201427007 b,Super Earth,2021,2.86,1.5,NaN


Now, let's return to the example of Kepler-97 c:

In [146]:
print(df.loc['Kepler-97 c'])

distance                      1308.0
star_mag                      12.994
planet_type                Gas Giant
discovery_yr                    2014
mass_ME                       343.44
radius_RE                           
orbital_radius_AU                NaN
orbital_period_yr                2.2
eccentricity                     0.0
detection_method     Radial Velocity
Name: Kepler-97 c, dtype: object


As you can see, there's a NaN in the orbital radius entry and a blank in the planet radius entry. Let's also double-check which columns have the whitespace filler problem:

In [116]:
print(df.dtypes)

distance             float64
star_mag             float64
planet_type           object
discovery_yr           int64
mass_ME               object
radius_RE             object
orbital_radius_AU    float64
orbital_period_yr    float64
eccentricity         float64
detection_method      object
dtype: object


The only unexpected types are for mass and radius, which should be floats, so let's go ahead and fix those:

In [117]:
df['mass_ME'] = df['mass_ME'].replace(' ', np.nan).astype('float64')
df['radius_RE'] = df['radius_RE'].replace(' ', np.nan).astype('float64')

Note that the `.replace()` method I used here is **not** the vector string method, but a Series/DataFrame method of the same name. We'll get to more Series and DataFrame methods later when we talk about operations.

Once all the white-spaces have been converted to NaN and the type is converted to float, it becomes possible to use `.isna()` and `.notna()`, and a number of other methods and function kwargs for handling NaNs become available:
- `df.fillna(fill_value, inplace=False)` systematically replaces NaNs or Nones with the specified fill value. If the data are object- or string-type, *fill_value* can be a string.
- `df.dropna(axis=axis, inplace=False)` drops rows (axis=0) or columns (axis=1) with missing values.
- If you need to interpolate over missing data, you can use `ser.interpolate(method=method)` where ser is a Series of numerical or time-like data and method can be 'linear', 'time', 'index', or a SciPy interpolation method. Technically you can use `interpolate()` on a DataFrame as well, but a DataFrame may not be the best format to store image-like data. If you need access-by-label capabilities for images and image cubes, the module you probably want is [Xarray](https://xarray.dev/).
- Many numerical methods like `.mean()` and `.cumsum()` have a `skipna` kwarg to control whether or not NaNs are included in the calculation (default is to skip them).

**Real-valued bad data.** If you have numerical data that you know to be bad (e.g. all those perfect 0's in the `eccentricity` column of the example DataFrame), or if there are infinities that you would prefer to be masked, there is the `df.mask(condition, other=None)` function, where you can pass conditional selection criteria and use the `other` kwarg to provide substitute values in the form of a scalar, Series, DataFrame, or callable.

In [120]:
print(df['eccentricity'].head())

#name
11 Comae Berenices b    0.23
11 Ursae Minoris b      0.08
14 Andromedae b         0.00
14 Herculis b           0.37
16 Cygni B b            0.68
Name: eccentricity, dtype: float64


In [121]:
print(df['eccentricity'].mask(df['eccentricity']==0.0).head())

#name
11 Comae Berenices b    0.23
11 Ursae Minoris b      0.08
14 Andromedae b          NaN
14 Herculis b           0.37
16 Cygni B b            0.68
Name: eccentricity, dtype: float64


### Reindexing, Sorting, Comparing, and Combining
Have more than one DataFrame? Do they have partially overlapping column or index labels? Do you worry that some data are duplicated across two DataFrames? No problem! Pandas has functions for a variety of methods to aggregate multiple DataFrames in whole or in part, and to compare two DataFrames with identical row/column labels. However, to do the comparisons and combine the data sensibly, two the indexes and column names of the two or more data structures to be compared or combined must be in a reproducible order, so I will start with sorting and reindexing.

#### Reindexing
If indexes & column names are missing or need to be updated, you can modify them in-place with `.reindex(index=rows, columns=cols)` (kwargs optional). You can also use `.reindex()` to select data where some index values used as args may not exist, without raising exceptions. There is a related `.reindex_like()` function that lets you copy the data from the DataFrame you call it on into a DataFrame with the row & column labels of the DataFrame you enter as the function arg.

There is a `.searchsorted(values)` Series method that returns indexes at which to insert values to maintain order, like NumPy's `.searchsorted()` version, but `.reindex()` makes it somewhat redundant.

In [95]:
dummy = pd.DataFrame(np.random.randint(0,high=9,
                                       size=(4,3)),
                     columns = ['a','b','c'],
                     index = [38,42,36,48])
print(dummy)
print(dummy.reindex(np.arange(36,50,2),
                    axis='index'))

    a  b  c
38  0  3  0
42  1  1  2
36  6  3  4
48  2  7  1
      a    b    c
36  6.0  3.0  4.0
38  0.0  3.0  0.0
40  NaN  NaN  NaN
42  1.0  1.0  2.0
44  NaN  NaN  NaN
46  NaN  NaN  NaN
48  2.0  7.0  1.0


In [96]:
dummy2 = pd.DataFrame(
    np.arange(0,9).reshape(3,3),
    columns = ['b','c','d'],
    index = [38,43,48])
print(dummy2.reindex_like(dummy))

     a    b    c
38 NaN  0.0  1.0
42 NaN  NaN  NaN
36 NaN  NaN  NaN
48 NaN  6.0  7.0


#### Sorting
Both DataFrames and Series can be sorted by index, by value, or some combination of both. In the case of DataFrames, row indexes and column labels can be sorted simultaneously. There are 2 methods to sort both Series & DataFrames: `.sort_values(by=row_or_col, axis=0, kind='quicksort')` & `.sort_index(axis=0)`.

In both cases, the axis given to the `axis` kwarg is the direction along which values will shift, not the fixed axis. Both return copies unless `inplace=True`, and both have a `key` kwarg that accepts a *vectorized* function (more on those shortly) to apply to the input index before sorting. The key usage as described in the official documentation is a bit misleading in its brevity: it does not alter what indexes are printed, but instead internally alters what indexes the sorting algorithm sees. For example if you have columns you want to sort alphabetically, but only some of the column names are capitalized, the .sort_index() function would normally place any column label starting with a lower-case letter after any column with a capitalized label. To treat capital and lower-case letters equally, you would have to give `key` a function that converts the whole list of indexes to lower-case letters, as I demonstrate below.

`.sort_values(by=row_or_col, kind='quicksort')` sorts Series or DataFrames by the values of the given column(s)/row(s) passed to the `by` kwarg (optional for Series). If `by` is one or more row indexes, it is mandatory to set `axis=1` or `axis='columns'`. If `by` is a list, the sorting order may depend on the algorithm given for the `kind` kwarg. I leave it as an exercise to the reader to look up various sorting methods to see how they might interact with the material. 

In [2]:
dummy = pd.DataFrame(np.random.randint(0,high=9,size=(4,3)),
                     columns = ['B','a','C'],
                     index = ['h','i','j','k'])
print("Sorted by column C\n",dummy.sort_values('C',axis=0))
print("Sorted by row j\n",dummy.sort_values('j',axis=1))

Sorted by column C
    B  a  C
k  8  3  1
i  2  1  7
j  6  8  7
h  0  1  8
Sorted by row j
    B  C  a
h  0  8  1
i  2  7  1
j  6  7  8
k  8  1  3


In [72]:
print("Columns sorted alphabetically\n",
      dummy.sort_index(axis=1))

Columns sorted alphabetically
    B  C  a
h  3  8  5
i  2  1  6
j  2  1  7
k  8  3  1


In [73]:
print("Columns sorted alphabetically with key\n",
      dummy.sort_index(axis=1,key=lambda c: c.str.lower()))

Columns sorted alphabetically with key
    a  B  C
h  5  3  8
i  6  2  1
j  7  2  1
k  1  8  3


#### Combining DataFrames
There are 4 Pandas functions and 3 DataFrame methods that can be used to combine 2 (or sometimes more) DataFrames (or Series). The Pandas functions are:
1. `.concat()`: combine 2 or more DataFrames or Series along a shared column or index, with optional set logic for handling other axes using the `join` kwarg.
2. `.merge(left_df, right_df, how='inner')`: combine 2 or more DataFrames with SQL-database-style join options specified using the `how` kwarg.
3. `.merge_ordered()`: combine 2 sorted DataFrames or Series with optional interpolation across gaps.
4. `.merge_asof()`: left-join 2 sorted DataFrames or Series by the nearest value of given index instead of requiring identical indexes. Typically used for time series.

And the DataFrame methods are:
1. `df1.combine_first(df2)`: update missing values of DataFrame `df1` with fill values from DataFrame `df2` at shared index locations, and add rows or columns from `df2` that did not exist in `df1`.
2. `df1.combine(df2, func)`: merge 2 DataFrames column-wise based on given function `func` that takes 2 Series & outputs either Series or scalars. Scalar outputs will be propagated to the whole column. The resulting DataFrame's row and column labels will be the union of the row and column labels of both DataFrames.
3. `df1.join(df2, on=[cols,or,inds])` (uses `.merge()` internally): join 2 DataFrames on given column(s) or index(es).

The easiest to understand is `.concat()`, so I'll start with that.

In [3]:
#start with dummy dataframe from before
print(dummy)

   B  a  C
h  0  1  8
i  2  1  7
j  6  8  7
k  8  3  1


In [106]:
#define second dummy with more predicatable values
dummy2 = pd.DataFrame(np.arange(0,12).reshape(4,3),
                        columns = ['B','a','C'],
                        index = ['q','r','s','t'])
print(dummy2)

   B   a   C
q  0   1   2
r  3   4   5
s  6   7   8
t  9  10  11


In [107]:
print(pd.concat((dummy.T,dummy2.T), axis=1))

   h  i  j  k  q  r  s   t
B  0  2  6  8  0  3  6   9
a  1  1  8  3  1  4  7  10
C  8  7  7  1  2  5  8  11


Before demonstrating `.merge()` and `.join()`, it will help to go over the meanings of the different SQL-style join options that you can pass to them with the `how` kwarg. The allowed options for the `how` kwarg are:
1. `'inner'` (default): take only the intersection of the 2 DataFrames in terms of their column headers and contents at specific row *positions*, like SQL `inner join`.
2. `'outer'`: align on any shared values at shared row and column indexes but keep all contents of both DataFrames, like SQL `full outer join`. The resulting DataFrame will have all non-rendundant permutations of the row and column indexes of both DataFrames, with NaNs inserted wherever those row and column combinations do not point to any existing data.
3. `'left'`: keep all contents of the left (first) DataFrame, plus any data from the right (second) that share row and column indexes from the left DataFrame, like SQL `left outer join`.
4. `'right'`: keep all contents of the right (second) DataFrame, plus any data from the left (first) that share row and column indexes from the right DataFrame, like SQL `right outer join`.
5. `'cross'`: take the Cartesian product\* of the two DataFrames, prioritizing the order of the left one, like SQL `cross join`.

\*If you don't know/remember what a Cartesian product is, imagine you have 2 sets of data, A=(x,y,z) and B=(1,2,3). The Cartesian product, A$\times$B, would then be the 9 ordered pairs in the following table:
|A$\times$B| 1 | 2 | 3 |
|---|---|---|---|
| **x** | (x,1) | (x,2) | (x,3) |
| **y** | (y,1) | (y,2) | (y,3) |
| **z** | (z,1) | (z,2) | (z,3) |


The quickest way to show what the first 4 cases mean intuitively is with the following Venn diagrams, where the contents of each circle are the unique combinations of row and column labels in each DataFrame ([source](https://www.ionos.co.uk/digitalguide/hosting/technical-matters/sql-outer-join/)):

In [204]:
from IPython.display import Image
Image(url='https://www.ionos.co.uk/digitalguide/fileadmin/DigitalGuide/Screenshots_2018/Outer-Join.jpg',
      width=600)

`Cross` joining is best demonstrated like this ([Source](https://www.sqlshack.com/sql-cross-join-with-examples/)):

In [205]:
Image(url='https://www.sqlshack.com/wp-content/uploads/2020/02/sql-cross-join-working-mechanism.png',
      width=600)


In all but the `'outer'` case, the  the order of indexes or keys in the first DataFrame is preserved unless the `sort` kwarg is True or other kwargs are used to manipulate the indexes. In the `'outer'` case, column indexes are sorted alpha-numerically. In most cases, row indexes/labels are not preserved with `merge()` but for `join()` they often are.

By default, `merge()` scans all columns with shared names across both DataFrames looking for rows where identical data share identical column names and row *positions* (not labels, but underlying 0-based numerical indexes), and aligns the DataFrames based on those intersections. If there are multiple possibilities, `inner ` returns all of those rows (and only those rows); `outer` returns all of the content aligned with those rows as much as possible with NaNs for padding; and `left` and `right` return the selected DataFrame with the columns of the other DataFrame with matching data appended at the rows where the data aligned, and NaNs in the rows without matching data.

If you only want to align on a subset of the shared data, you can provide column labels to the `on` kwarg, but the merger will only succeed if **both** of the following conditions are met:
1. The column labels provided exist in both DataFrames.
2. Both DataFrames have the same data in the specified columns *at the same row positions*.

You can also specify `left|right_on` alignments separately, and/or `left|right_index=True` to align on row indexes from the left or right DataFrame, but these can be very tricky to use successfully. The examples below will clarify some of these details, but it would take much more time and verbiage than either I or even the official Pandas documentation can demonstrate how all of the different kwargs interact with each other and with varying sizes of DataFrames.

In [210]:
#(re)define more orderly dummy dataframes as needed for examples
dummy2 = pd.DataFrame(np.arange(0,12).reshape(4,3),
                        columns = ['A','B','C'],
                        index = ['e','f','g','h'])
dummy3 = pd.DataFrame(np.arange(-5,11).reshape(4,4),
                        columns = ['B','C','D', 'E'],
                        index = ['f','g','h','i'])
dummy3.loc['g',['B','C']] = [1,2]

In [212]:
#demonstrate 'outer', 'inner', 'left', 'right', 'cross'
print(dummy2,'\n')
print(dummy3,'\n')
print(pd.merge(dummy2,dummy3, how='cross'))#, on=['B','C']))

   A   B   C
e  0   1   2
f  3   4   5
g  6   7   8
h  9  10  11 

   B  C  D   E
f -5 -4 -3  -2
g  1  2  1   2
h  3  4  5   6
i  7  8  9  10 

    A  B_x  C_x  B_y  C_y  D   E
0   0    1    2   -5   -4 -3  -2
1   0    1    2    1    2  1   2
2   0    1    2    3    4  5   6
3   0    1    2    7    8  9  10
4   3    4    5   -5   -4 -3  -2
5   3    4    5    1    2  1   2
6   3    4    5    3    4  5   6
7   3    4    5    7    8  9  10
8   6    7    8   -5   -4 -3  -2
9   6    7    8    1    2  1   2
10  6    7    8    3    4  5   6
11  6    7    8    7    8  9  10
12  9   10   11   -5   -4 -3  -2
13  9   10   11    1    2  1   2
14  9   10   11    3    4  5   6
15  9   10   11    7    8  9  10


For the `on` kwarg, you should try to avoid providing more than one column at a time unless you know for sure the values in both DataFrames are the same at those columns *and in the same rows*. In the above example, I can use `['B','C']` with `how='inner'` only because in both DataFrames there exists a row with a 7 in column `B` *and* an 8 in column `C`, i.e. row `g` of the first DataFrame, and row `i` of the second. Therefore, the inner merge result of `pd.merge(dummy2,dummy3, on=['B','C'], how='inner')` is the union of those two rows, which in turn are the intersection of the two DataFrames. In fact, this intersection is the only possible point of alignment given the (original) contents of these DataFrames, so the inner and outer merge results are both unchanged by leaving out `on=['B','C']'`. It would be a different story if I, say, changed row `g` of `dummy3` to `[1, 2, 1, 2]`.

I would recommend avoiding any case where you have to use `left|right_index=True` and/or `left|right_on` kwargs, because it quickly becomes difficult to determine which combinations will not raise a `MergeError` without, e.g., working it out on paper.

In [105]:
dummy2.join(dummy3,lsuffix='_l',rsuffix='_r')
#merge uses x & y by default, but join has empty defaults

,A,B_l,C_l,B_r,C_r,D,E
e,0,1,2,NaN,NaN,NaN,NaN
f,3,4,5,-5.0,-4.0,-3.0,-2.0
g,6,7,8,1.0,2.0,1.0,2.0
h,9,10,11,3.0,4.0,5.0,6.0


Now I'll demonstrate `.combine_first()` and `.combine()`. The former is a bit more straightforward.

In [126]:
dummy3.mask(dummy3<=1, inplace=True) #create (more) missing values to replace
print(dummy3)

     B    C    D     E
f  NaN  NaN  NaN   NaN
g  NaN  2.0  NaN   2.0
h  3.0  4.0  5.0   6.0
i  7.0  8.0  9.0  10.0


In [127]:
dummy3.combine_first(dummy2)

,A,B,C,D,E
e,0.0,1.0,2.0,NaN,NaN
f,3.0,4.0,5.0,NaN,NaN
g,6.0,7.0,2.0,NaN,2.0
h,9.0,3.0,4.0,5.0,6.0
i,NaN,7.0,8.0,9.0,10.0


Notice that `dummy3.combine_first(dummy2)` not only replaced the NaNs in `dummy3` with numeric values from `dummy2` at the same row and column locations, but also prepended column `A` and row `e` to `dummy3`.

`.combine()` is a little bit trickier because it relies on you providing a sensible function to handle locations where both DataFrames have data. Functions may be user-defined (named or passed as lambda functions), built-in, or provided by other imported modules like NumPy, but they must accept two 1D arrays and return either a 1D array or a scalar. Functions that return scalars will propagate the same value to the whole column. Once again, I'm starting with that masked `dummy3` DataFrame, and I will show the results of a couple different functions. The first version is a mistake you might make if you're trying to make `combine()` take the higher value at any position where both DataFrames have data.

In [182]:
#print these to refresh our memories
print(dummy3,'\n')
print(dummy2,'\n')
#print the first way I tried to code up the function
# that didn't raise a type error
dummy3.combine(dummy2, lambda x,y: np.nanmax([x,y]))

     B    C    D     E
f  NaN  NaN  NaN   NaN
g  NaN  NaN  NaN   2.0
h  3.0  4.0  5.0   6.0
i  7.0  8.0  9.0  10.0 

   A   B   C
e  0   1   2
f  3   4   5
g  6   7   8
h  9  10  11 



,A,B,C,D,E
e,9.0,10.0,11.0,9.0,10.0
f,9.0,10.0,11.0,9.0,10.0
g,9.0,10.0,11.0,9.0,10.0
h,9.0,10.0,11.0,9.0,10.0
i,9.0,10.0,11.0,9.0,10.0


The following is what is actually required to achieve the desired result without learning any new methods:

In [183]:
def elem_wise_nanmax(x,y):
    rns = set(list(x.index)+list(y.index)) #get union of row indexes
    ser = pd.Series(index=rns) #create Series to store results for return
    for i in list(rns): #iterate over rows
        try: #take nanmax if values exist in both input series...
            ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])
        except IndexError: #...otherwise take whichever one exists
            if i in x.index:
                ser.loc[i]=x.loc[i]
            elif i in y.index:
                ser.loc[i]=y.loc[i]
    return ser # return results in series form

dummy3.combine(dummy2,elem_wise_nanmax) #combine applies elem_wise_nanmax column-by-column

/tmp/ipykernel_10410/776801286.py:6: RuntimeWarning: All-NaN axis encountered
  ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])
/tmp/ipykernel_10410/776801286.py:6: RuntimeWarning: All-NaN axis encountered
  ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])
/tmp/ipykernel_10410/776801286.py:6: RuntimeWarning: All-NaN axis encountered
  ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])


,A,B,C,D,E
e,0.0,1.0,2.0,NaN,NaN
f,3.0,4.0,5.0,NaN,NaN
g,6.0,7.0,8.0,NaN,2.0
h,9.0,10.0,11.0,5.0,6.0
i,NaN,7.0,8.0,9.0,10.0


As you can see, this is pretty clunky, and iteration makes this way slower than necessary. Fortunately, Pandas provides ways of broadcasting functions over rows and/or columns simultaneously, without iteration. So, let's get into Pandas functions!

## Operations
Pandas DataFrames have method counterparts to most NumPy statistical functions, string methods, plus **many** other convenience functions. We've already covered functions that report basic descriptive data about the DataFrame and functions that handle NaNs. Now we'll focus on the functions that move, change, or do something else with valid data or the Series/DataFrame as a whole. 

#### Comparisons
You can compare a Series or a DataFrame to a single scalar value with normal operators (`>=`, `!=`, etc).
To compare 1 Series or DataFrame element-wise to another of the same shape (as the arg), use `.gt()`, `.lt()`, `.ge()`, `.le()`, `.eq()`, or `.ne()`. For any of the aforementioned comparison methods (to scalars or other pandas data structures), you can add `.any()` or `.all()` once to collapse the column axis, or twice to get 1 value. This is called Boolean Reduction.

To find & print differences between 2 identically indexed Series or DataFrames (let's call them `df1` and `df2`; both objects must have the same row & column labels in the same order), you can use `df1.compare(df2)`. The `.compare()` method will show differences as subtle as extraneous whitespaces or differences of capitalization. However, it will **not** show data type differences if the values are numerically equal; for that, you''l need to use `pd.testing.assert_frame_equal(df1, df2)` or `pd.testing.assert_series_equal(df1, df2)` to see if it raises `AssertionError`.

In [21]:
dummy = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns = ['a','b','c'], index = [0,1,2])
dummy2 = dummy.astype(float)
print(dummy.compare(dummy2))
print(pd.testing.assert_frame_equal(dummy,dummy2,check_dtype=False)) #assertion error if True

Empty DataFrame
Columns: []
Index: []
None


#### Statistics
Stats methods (`.mean()`, `.std()`, `.median()`, `.min()`/`.max()`, `.cumsum()`/`.comprod()`, `cov()`, `corr()`, etc.) mostly work like you'd expect based on their NumPy counterparts, and also ignore NaNs by default. If you input a Series or one column of a DataFrame, the result is typically a single value or a Series of the same length. If you input a DataFrame, you will have to choose an axis to evaluate the function over, and the result will be either a Series or, as in the case of the `.corr()` function, a DataFrame in the form of a square matrix with identical row and column labels.

In [206]:
print('The average detected exoplanet mass is {:.3f} \
Jupiter masses.'.format(df['mass_ME'].mean()/317.907))

The average detected exoplanet mass is 1.447 Jupiter masses.


In [205]:
print('The most distant planet detected is \
{:.0f} light years away.'.format(df['distance'].max()))

The most distant planet detected is 27727 light years away.


In [334]:
print("The standard deviation and standard error of the mean for\
 orbital radii are {:.0f} and {:.0f} AU, respectively.\
 ".format(df['orbital_radius_AU'].std(), df['orbital_radius_AU'].sem()))

The standard deviation and standard error of the mean for orbital radii are 139 and 2 AU, respectively. 


In [317]:
print(df['planet_type'].mode()) #output is a Series
print("The most common type of exoplanet detected is \
{}.".format(df['planet_type'].mode()[0]))

0    Neptune-like
Name: planet_type, dtype: object
The most common type of exoplanet detected is Neptune-like.



There are also methods that you typically need SciPy for, like `.skew()` and `.kurtosis()`, and functions for assigning ranks (`.rank()`) and quantiles (`.quantile()`).

All of the above can be applied to the whole DataFrame column-wise or row-wise, and with the `.rolling` attribute between the DataFrame and the function, can also be evaluated over rolling windows of $n$ entries instead of the whole axis.

#### Binary Operations

The more exciting features of Pandas are that you can broadcast operations across rows, columns, whole DataFrames, and even do arithmetic on multiple DataFrames. For these purposes, DataFrames have the methods `add()`, `sub()`, `mul()`, `div()`, `pow()`, `mod()`, and `divmod()`, and reversed versions of each (`radd()`, `rsub()`, ...) for if the operations do not commute. To take `div()` for example, the forward version `dfA.div(other)` divides `dfA` by `other`, where `other` could be a scalar, a Series or vector of the same length as one of the dimensions of `dfA`, or a DataFrame of the same shape as `dfA`. In the reverse version, `dfA.rdiv(other)` divides `other` by `dfA`.

Remember that to divide a DataFrame by a vector or Series, you must specify an axis. The axis determines which dimension the operation is to be broadcast along, NOT the orientation of the vector. If you set `axis='index'` or 0, each entry in the vector will be paired with a row, and if you set `axis='columns'`, each entry in the vector will be matched to a column.

In [22]:
dfA = pd.DataFrame(np.arange(12).reshape([4,3]),
                   columns = ['a','b','c'])
print(dfA,'\n\n',dfA.div([4.,3.,2., 1.], axis='index'),
      '\n\n', dfA.rdiv([4.,3.,2., 1.], axis='index'))

   a   b   c
0  0   1   2
1  3   4   5
2  6   7   8
3  9  10  11 

      a          b          c
0  0.0   0.250000   0.500000
1  1.0   1.333333   1.666667
2  3.0   3.500000   4.000000
3  9.0  10.000000  11.000000 

           a         b         c
0       inf  4.000000  2.000000
1  1.000000  0.750000  0.600000
2  0.333333  0.285714  0.250000
3  0.111111  0.100000  0.090909


The method `.divmod()` is a bit special in that a) it can only be called on dividends of Series or Index type, b) and it returns 2 Series instead of just 1. The first Series is the results of floor division, and the second Series is the list of remainders. The divisor (the argument in parentheses) may be either a scalar or a Series of the same length as the dividend.

In [10]:
ser3 = pd.Series(np.linspace(0,9,19))
rems,mods=ser3.divmod(3)
print(pd.concat((ser3.rename('nums'),
                 rems.rename('rems'),
                 mods.rename('mods')), axis=1))

    nums  rems  mods
0    0.0   0.0   0.0
1    0.5   0.0   0.5
2    1.0   0.0   1.0
3    1.5   0.0   1.5
4    2.0   0.0   2.0
5    2.5   0.0   2.5
6    3.0   1.0   0.0
7    3.5   1.0   0.5
8    4.0   1.0   1.0
9    4.5   1.0   1.5
10   5.0   1.0   2.0
11   5.5   1.0   2.5
12   6.0   2.0   0.0
13   6.5   2.0   0.5
14   7.0   2.0   1.0
15   7.5   2.0   1.5
16   8.0   2.0   2.0
17   8.5   2.0   2.5
18   9.0   3.0   0.0


#### Vectorized String Methods
Most familiar string methods like `.replace()`, `.lower()`/`.upper()`, `.split()`/`.rsplit()`, and `.strip()` have counterparts in Pandas that can be broadcast to every element with the syntax `.str.<method>`. The available args and kwargs for each of the functions are also mostly unchanged, except in the case of `.str.replace()`: the standard two positional arguments are accepted, as are regular expressions, but it will **not** accept a dictionary where the keys are the existing substrings and the values are the replacement characters. (Note: dictionary replacement also fails for the built-in string `replace()` method if any of the old or new strings contain LaTeX expressions because `replace()` implicitly uses string insertion via `'{}'.format()`, which collides with LaTeX's use of curly braces as delimiters for function arguments.)

The function `.str.split(" ", expand=False, n=None)` and its `.str.rsplit()` counterpart return either a Series in which the items are lists or, with the kwarg `expand=True`, a DataFrame with as many columns as there would have been items in the longest list of substrings (shorter lists will leave Nones in the columns they weren't long enought to fill). If you want to limit the number of length of each list or row, you can set `n` equal to the maximum number of occurences of the character or substring to split on. Unfortunately, if you set `expand=True`, there is not currently a kwarg to set the column names of the resulting DataFrame in the same step, so at least initially both rows and columns will be labeled with 0-based indexes. With `expand=False` (default), you can also use either `.str.get(i)` or just `.str[i]`, where `i` is the list index(es) or slice, to return a Series of subsets of each list of split strings. I'll demonstrate:

In [187]:
dummy4 = pd.Series(['Pandas are cute!', 'I like trains.','Hello, world?', 'Hams.'])
print('Original:\n', dummy4, '\n')
dummy5 = dummy4.str.split()
print('Split and get:\n',dummy5.str[:2], '\n')
dummy6 = dummy4.str.split(expand=True)
print('Expand=True:\n', dummy6)

Original:
 0    Pandas are cute!
1      I like trains.
2       Hello, world?
3               Hams.
dtype: object 

Split and get:
 0       [Pandas, are]
1           [I, like]
2    [Hello,, world?]
3             [Hams.]
dtype: object 

Expand=True:
         0       1        2
0  Pandas     are    cute!
1       I    like  trains.
2  Hello,  world?     None
3   Hams.    None     None


#### Time Series Functions

### User-Defined Methods
If you find that you need to broadcast more complicated functions to your data, there are 4 methods you can use depending on whether the function is a combination of named NumPy functions or totally user-defined, what shape the function broadcasts or reduces (aggregates) results to, and /or whether the function is to be applied row-wise, column-wise, or element-wise.

#### Aggregating (aka reducing) with `.agg()`
The `.agg()` method only accepts functions that take all the values along a specified axis (row or column in this case) as input and then output a single value, like `sum()` or `std()`. You can supply more than one of these functions at a time as a list of function names.

#### Broadcasting with `.transform()`
The `.transform()` method requires the output to have the same shape as the input, but you can apply more than one function at a time as long as the functions are passed by name. You could even set up a function list to apply a different function to every column of a DataFrame. The powerful feature of `.transform()` is that it can be applied group-wise to GroupBy objects

#### Elementwise functions with `.map()`
The `.map()` method is like a more restricted version of transform, and is typically used for element-wise replacement of values by function. It can only take 1 function at a time, but it is typically cleaner and more intuitive if that's all you need.

That said, if you can recast your desired operations as one or more vectorized functions, like `df**0.5` instead of `df.map(np.sqrt)`, the vectorized version will generally be faster.

One simple use case is if your function is piecewise.

In [9]:
def my_func(T):
    if T<=0 or np.isnan(T) is True:
        pass
    elif T<300:
        return 0.2*(T**0.5)*np.exp(-616/T)
    elif T>=300:
        return 0.9*np.exp(-616/T)
    
junk = pd.DataFrame(np.random.randint(173,high=675,size=(4,3)),
                    columns = ['A', 'B', 'C'])
print(junk,'\n')
print(junk.map(my_func))

     A    B    C
0  231  426  572
1  497  628  410
2  375  600  577
3  408  206  616 

          A         B         C
0  0.211211  0.211957  0.306578
1  0.260593  0.337479  0.200328
2  0.174117  0.322379  0.309452
3  0.198858  0.144310  0.331091


#### For most other operations, there's `.apply()`

Remember waaaaay back to when we were trying to combine 2 DataFrames and take the higher value wherever both DataFrames had data? We can do that faster and with less code using `.apply()`. I'll repost the dummy DataFrames and the clunky function that let us combine them as desired, and then show the improved way of doing it with `.apply()`.

In [179]:
dummy2 = pd.DataFrame(np.arange(0,12).reshape(4,3),
                        columns = ['A','B','C'],
                        index = ['e','f','g','h'])
dummy3 = pd.DataFrame(np.arange(-5,11).reshape(4,4),
                        columns = ['B','C','D', 'E'],
                        index = ['f','g','h','i'])
dummy3.mask(dummy3<=1, inplace=True) #create more missing values to replace
print(dummy2,'\n')
print(dummy2,'\n')

   A   B   C
e  0   1   2
f  3   4   5
g  6   7   8
h  9  10  11 

   A   B   C
e  0   1   2
f  3   4   5
g  6   7   8
h  9  10  11 



In [201]:
import time
t0 = time.time()
# Old clunky function:
def elem_wise_nanmax(x,y):
    rns = set(list(x.index)+list(y.index))
    ser = pd.Series(index=rns)
    for i in list(rns):
        try:
            ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])
        except IndexError:
            if i in x.index:
                ser.loc[i]=x.loc[i]
            elif i in y.index:
                ser.loc[i]=y.loc[i]
    return ser

print(dummy3.combine(dummy2,elem_wise_nanmax))
print("time taken in s:", time.time()-t0)

     A     B     C    D     E
e  0.0   1.0   2.0  NaN   NaN
f  3.0   4.0   5.0  NaN   NaN
g  6.0   7.0   8.0  NaN   2.0
h  9.0  10.0  11.0  5.0   6.0
i  NaN   7.0   8.0  9.0  10.0
time taken in s: 0.010122537612915039


/tmp/ipykernel_10410/3482005737.py:9: RuntimeWarning: All-NaN axis encountered
  ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])
/tmp/ipykernel_10410/3482005737.py:9: RuntimeWarning: All-NaN axis encountered
  ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])
/tmp/ipykernel_10410/3482005737.py:9: RuntimeWarning: All-NaN axis encountered
  ser.loc[i] = np.nanmax([x.loc[i],y.loc[i]])


In [203]:
# new hotness (less intuitive & still clunky, but faster)
t0 = time.time()
def elem_wise_nanmax(row,dfr): #row is from df_left, dfr is df_right
    row2 = dfr.loc[row.name] if row.name in dfr.index else pd.Series(index=dfr.columns)
    return row.combine(row2, lambda x, y: np.nanmax([x, y]))        
#.name attribute returns the name of a column if there is one, otherwise it returns a row name
print(dummy3.combine_first(dummy2).apply(lambda z: elem_wise_nanmax(z,dummy2), axis=1))
print("time taken in s:", time.time()-t0)

     A     B     C    D     E
e  0.0   1.0   2.0  NaN   NaN
f  3.0   4.0   5.0  NaN   NaN
g  6.0   7.0   8.0  NaN   2.0
h  9.0  10.0  11.0  5.0   6.0
i  NaN   7.0   8.0  9.0  10.0
time taken in s: 0.006251811981201172


/tmp/ipykernel_10410/2096513521.py:5: RuntimeWarning: All-NaN axis encountered
  return row.combine(row2, lambda x, y: np.nanmax([x, y]))
/tmp/ipykernel_10410/2096513521.py:5: RuntimeWarning: All-NaN axis encountered
  return row.combine(row2, lambda x, y: np.nanmax([x, y]))


## Preparing input for Machine Learning (ML)
ML programs like TensorFlow and PyTorch are highly compatible with Series/DataFrame inputs, but there are still limitations to the data types that will produce useful results. For example (and this doesn't make much sense given what we actually know about exoplanets, but) let's say you wanted to build a neural network model that predicts the planet type based on a subset of the physical or orbital parameters. The physical and orbital parameters of the existing data are numerical, but Planet Type is a categorical variable with 5 unique values. What we would want to do, then, is turn each of those categories into columns where the 

## GroupBy Objects

GroupBy objects (more properly DataFrameGroupBy) are the data returned by the function `df.groupby(by, axis=0)`, where the `by` argument can be a column label, a list of column names, row index(es) if you set the axis kwarg equal to 1, or even a Series. There is also a level kwarg if you choose to call `groupby()` on a hierarchical DataFrame... 

[need to add a bunch of stuff here. refer to](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

When inspecting a GroupBy object, many DataFrame inspection methods still work, and there is an added `.nth()` method that lets you grab the *n*th row of every group, where *n* is a 0-based index that can also be negative if you want to start from the bottom. For instance, if you wanted just the last row of every group in a DataFrame `df` grouped by some column `col_B`, `df.groupby('col_B').nth(-1)` would return the same rows as `df.groupby('col_B').tail(1)`.

...

GroupBy objects share some built-in functions with DataFrames, but that does not include binary math functions. You should also be wary of using `.apply()` on a GroupBy object because `apply()` has to infer the level at which it's supposed to start applying the function, and that doesn't always work correctly.

In [7]:
grouped1=df.groupby(['planet_type'])
grouped1.nth(-1)

,distance,star_mag,planet_type,discovery_yr,mass_ME,radius_RE,orbital_radius_AU,orbital_period_yr,eccentricity,detection_method
#name,,,,,,,,,,
LkCa 15 c,516.0,12.025,Unknown,2015,NaN,NaN,18.60000,0.999316,0.00,Direct Imaging
Wolf 503 b,145.0,10.270,Neptune-like,2018,6.26,2.043,0.05706,0.016427,0.41,Transit
YSES 2 b,357.0,10.885,Gas Giant,2021,2003.40,12.768,115.00000,1176.500000,0.00,Direct Imaging
YZ Ceti b,12.0,12.074,Terrestrial,2017,0.70,0.913,0.01634,0.005476,0.06,Radial Velocity
YZ Ceti d,12.0,12.074,Super Earth,2017,1.09,1.030,0.02851,0.012868,0.07,Radial Velocity


In [9]:
grouped1['orbital_radius_AU'].mean()

planet_type
Gas Giant       21.515449
Neptune-like     0.224902
Super Earth      0.109952
Terrestrial      0.062381
Unknown         16.650000
Name: orbital_radius_AU, dtype: float64

In [16]:
grouped1.get_group('Unknown').iloc[:,6:8]

,orbital_radius_AU,orbital_period_yr
#name,,
KIC 10001893 b,NaN,0.000548
KIC 10001893 c,NaN,0.000821
KIC 10001893 d,NaN,0.002190
LkCa 15 b,14.7,0.999316
LkCa 15 c,18.6,0.999316


In [36]:
grouped2=df.groupby(['detection_method'])
grouped2[['mass_ME','radius_RE', 'orbital_radius_AU', 'orbital_period_yr']].agg('mean')

,mass_ME,radius_RE,orbital_radius_AU,orbital_period_yr
detection_method,,,,
Astrometry,4890.840000,12.600000,0.499825,0.726626
Direct Imaging,7929.949333,15.835680,514.123769,40445.285440
Disk Kinematics,795.000000,13.216000,130.000000,957.300000
Eclipse Timing Variations,2154.773529,12.880000,3.962357,9.628240
Gravitational Microlensing,746.775584,10.241521,2.541477,7.065273
Orbital Brightness Modulation,350.513333,9.623000,0.013667,0.003164
Pulsar Timing,205.652857,5.395333,4.897800,17.617327
Pulsation Timing Variations,2385.000000,12.712000,1.700000,2.750000
Radial Velocity,1041.315930,10.031391,2.112706,5.167191


### Advanced I/O Example

To show you an output example in the HDF5 format, I'm going to add another column with the planet masses in the other typical mass unit, Jupiter masses, and then pack the whole table and a meta-data dictionary with the above definitions into an HDF5 file.

Jupiter's mass is 317.9 times that of Earth, so all I have to do is multiply the existing mass column by that number.